# Malaria Detection In Human Cells

## 1. Download Dataset

#### Used Kaggle Json to directly access kaggle dataset on google colab notebook without downloading it on our computer

In [ ]:
!ls -lha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

#### Uncompressing zip file

In [ ]:
from zipfile import ZipFile
print("uncompressing zip file")

filename = "cell-images-for-detecting-malaria.zip"

with ZipFile(filename, 'r') as zip:
    zip.extractall()

print("done")